<a href="https://colab.research.google.com/github/DelMashiry-dev/DelMashiry-dev/blob/main/REALTIMEPREDICTION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mount the Drive Containing the folder with the dataset


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!ls /content/drive/MyDrive/Medical_Resource_Prediction

In [ ]:
import pandas as pd
# Read first 100000 rows but only specific columns
df = pd.read_csv('/content/drive/MyDrive/Medical_Resource_Prediction/owid-covid-data.csv', nrows=100000, usecols=['iso_code', 'continent', 'location', 'date','total_cases', 'new_cases', 'total_deaths', 'new_deaths', 'reproduction_rate',
    'icu_patients', 'hosp_patients', 'weekly_icu_admissions', 'weekly_hosp_admissions','total_vaccinations', 'people_vaccinated', 'people_fully_vaccinated', 'new_vaccinations',
    'total_tests', 'new_tests', 'positive_rate','population', 'population_density', 'median_age', 'aged_65_older', 'aged_70_older','cardiovasc_death_rate', 'diabetes_prevalence'])
print(df.head())